# proyecto integrado 1

**Información del archivo**

En este proyecto se trabajarán datos de plataformas de videojuegos, se modificarán y se realizarán análisis basandose en los datos obtenidos.

**Paso 1. Abrir el archivo de datos y estudiar la información general**

In [ ]:
import pandas as pd #Importo las librerías
import numpy as np
import scipy.stats as st
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
games = pd.read_csv('/datasets/games.csv') #Cargo el dataframe
games.info() 

In [ ]:
games.sample(5, random_state = 1000)

In [ ]:
games.describe()

Puedo ver que es un archivo de plataformas de videojuegos, cuenta con 11 columnas, donde aparecen las ventas por los continentes de asia, europa y norteamérica, año de lanzamiento y nombre de los juegos, entre otros, tipo de datos de las columnas, veo que valores ausentes.

**Paso 2: Preparado de datos**

In [ ]:
games.columns = games.columns.str.lower() #convetir a minúscula los nombres de las columnas
print(games.columns)

In [ ]:
100*games.isna().sum()/games.shape[0] #verificar si hay valores nulos 

In [ ]:
games['name'] = games['name'].fillna('unknown') #Reemplazar valores ausentes por 'unknown' en la columna name, genre y rating
games['genre'] = games['genre'].fillna('unknown')
games['rating'] = games['rating'].fillna('unknown')

games.isna().sum()

In [ ]:
games.user_score.unique() #La columna user_score contiene valores tbd lo que dificulta modificarla

In [ ]:
games['user_score'] = games['user_score'].replace('tbd', np.nan) #reemplazo tbd por nan
games['user_score'] = games['user_score'].astype(float) #Ahora si puedo trabajar con esta columna
games.user_score.unique()

In [ ]:
games.duplicated().sum() #verifico duplicados

**Describe las columnas en las que los tipos de datos han sido cambiados y explica por qué.**

Al empezar a revisar el dataframe (df), se ve que la columna user_score se encuentra como object siendo una columna float, también se ven que poseen las columnas valores ausentes.

Comienzo pasando a minusculas los nombres de las columnas, verifico el porcentaje de valores nulos que hay por columna. Convierto los valores ausentes por 'unknown' en las columnas object ya que no corro riesgo de que ocurra sesgo de datos, sin embargo las columnas con dígitos las mantengo ya que los valores nan no impiden que pueda trabajar con los datos. Al tratar de convertir la columna user_score a float no se podía ya que la columna poseía valores 'tbd' (por determinar) asi que para manejar esto, los valores tbd los converti a nan, luego, a float.


In [ ]:
games['venta_total'] = games['na_sales'] + games['eu_sales'] + games['jp_sales'] #Calculo de ventas totales

games.head()

**Paso 3: Analisis de datos**

**3.1 Mira cuántos juegos fueron lanzados en diferentes años. ¿Son significativos los datos de cada período?**

In [ ]:
count_games = games.groupby('year_of_release')['name'].count().reset_index()
display(count_games) # Mira cuántos juegos fueron lanzados en diferentes años.

In [ ]:
a = 'Juegos vendidos por año'  

count_games = games.groupby('year_of_release')['name'].count().reset_index()

# gráfico de línea
plt.figure(figsize=(12, 8))
count_games.plot(x = 'year_of_release', y = 'name')
plt.title(a)
plt.xlabel('Año de Lanzamiento')
plt.ylabel('Número de Juegos Vendidos')
plt.show()


Segun los datos del df, ha habido un incremento con los años en la venta de videojuegos, siendo su pick entre 2002 y 2011, a partir de esos años ha ido disminuyendo la venta pero continúa siendo mayor a lo que se vendía en los años 80's y 90's.

**Paso 3.2** Venta por plataforma

**Observa cómo varían las ventas de una plataforma a otra. Elige las plataformas con las mayores ventas totales y construye una distribución basada en los datos de cada año.**

In [ ]:
sales_by_console = games.pivot_table(index = 'platform', values = 'venta_total', aggfunc = 'sum')
sales_by_console.sort_values(by = 'venta_total', ascending = False).head(5)

In [ ]:
platform_list = ('PS2', 'X360', 'Wii', 'PS3', 'DS') #creo una lista con las plataformas que más venden
games_2 = games[games['platform'].isin(platform_list)] #creo un df_2 con la lista que creé

In [ ]:
platform_distribution = games_2.groupby(['year_of_release', 'platform'])['venta_total'].sum().reset_index()
platform_distribution

Observando por separado estas plataformas:
se puede ver comenzando por 'DS' que hay un periodo entre los años 1985 y 2004 que no tiene ventas. Desde 2004 comienza a tener ventas >100, hasta 2010 que empiezan a decaer, el 2013 es el último registro que hay de 1.42.

La plataforma 'PS2' inicia el año 2000 con venta de 35.59, continúa con ventas >100 del 2001-2005, del 2006 comienzan a disminuir las ventas, llegando a lo minimo de ventas 2010-2011 (4.44-0.35)

La plataforma X360 inició en 2005 con bajas ventas, a partir del 2006-2013 tienen ventas que empienzan a decaer nuevamente en 2014 hasta ahora (2016).
Con Wii inicia con grandes ventas (2006), y empiezan a decaer fuertemente en 2011.





In [ ]:
sns.lineplot(data = platform_distribution, hue = 'platform', x = 'year_of_release', y = 'venta_total' )

3.3 **Busca las plataformas que solían ser populares pero que ahora no tienen ventas.**


In [ ]:
games.columns

In [ ]:
plat_by_year_min = games.groupby('platform').agg({'year_of_release': 'min', 'venta_total': 'sum'}).reset_index()
plat_by_year_min = plat_by_year_min.sort_values(by= 'year_of_release')
print('plataformas que solian ser populares:')
display(plat_by_year_min) #agrupo por el año menor que es el año que se lanzó la plataforma

In [ ]:
plat_by_year_max = games.groupby('platform').agg({'year_of_release': 'max', 'venta_total': 'sum'}).reset_index()
plat_by_year_max = plat_by_year_max.sort_values(by = 'year_of_release')
print('plataformas actualmente populares')
display(plat_by_year_max)  #agrupo por el año máximo que correspondería al último año que hay registros de esa plataforma

In [ ]:
# Calculo la diferencia entre los años de lanzamiento y las ventas totales
difference_df = plat_by_year_max[['platform', 'year_of_release', 'venta_total']].copy()
difference_df['year_of_release_diff'] = plat_by_year_max['year_of_release'] - plat_by_year_min['year_of_release']

# Mostrar el DataFrame con las diferencias
print('Diferencias entre plataformas que solían ser populares:')
display(difference_df)


mean_difference = difference_df['year_of_release_diff'].mean()
print('Media que duran las compañias:')
print(mean_difference)

In [ ]:
juegos_lanzados = games.groupby('platform')['name'].count()
display(juegos_lanzados.sort_values(ascending = False))



**¿Cuánto tardan generalmente las nuevas plataformas en aparecer y las antiguas en desaparecer?**

Aparecen nuevas plataformas cada 2-3 años pero plataformas que tengan realmente exito aparecen cada 4-5 años aprox revisando los datos, por otro lado en promedio las plataformas antiguas tardan en desaparecer 7-8 años aprox, se pudo ver que hay compañias que continuan existiendo como DS, PC, PS2 entre otras y puede deberse a que han ido renovándose y actualizando sus juegos y sus productos.

3.4 **Determina para qué período debes tomar datos. Para hacerlo mira tus respuestas a las preguntas anteriores. Los datos deberían permitirte construir un modelo para 2017**


El periodo que es mejor tomar datos es del 2000-2013, ya que a partir del 2000's hubo un aumento en la venta de estos juegos y luego del 2013 se puede ver una disminución en la venta de estos.

3.5 **Trabaja solo con los datos que consideras relevantes. Ignora los datos de años anteriores.** 
**¿Qué plataformas son líderes en ventas? ¿Cuáles crecen y cuáles se reducen? Elige varias plataformas potencialmente rentables.**


Las plataformas lideres en ventas actualmente (2016) son: Wii, PS3, X360, creo que estas plataformas continuarán teniendo ventas rentables los próximos años, todo depende si continúan innovando en sus juegos y tecnología.
Las plataformas que podrian estarse reduciendo: PSV, DS, WiiU, DC debido a que han bajado sus ventas, algunas no han sacado juegos del 2008-2013


6. **Crea un diagrama de caja para las ventas globales de todos los juegos, desglosados por plataforma. ¿Son significativas las diferencias en las ventas? ¿Qué sucede con las ventas promedio en varias plataformas? Describe tus hallazgos.**

In [ ]:
plt.figure(figsize=(12, 8))
sns.boxplot(data=games, x='venta_total', y='platform', showfliers=False)
plt.xticks(rotation=45, ha='right')
plt.show()

En el gráfico de caja puedo ver que hay datos sesgados, las ventas promedio de todas las plataformas varian, son signifícativas las diferencias ya que demuestran que a pesar de durar pocos años o lanzar menos juegos fueron teniendo significativas ventas en el periodo que estuvieron activas, ejemplo: GEN que lanzó 29 juegos y estuvo activa 1990-1994 o GB que estuvo 1988-2001 y lanzó 98 juegos.

In [ ]:
games.columns

**3.7 Mira cómo las reseñas de usuarios y profesionales afectan las ventas de una plataforma popular (tu elección).**

In [ ]:
wii_data = games[games['platform'] == 'Wii']
critic_game = wii_data.groupby('critic_score')['venta_total'].sum().reset_index()
display(critic_game.sample(10, random_state= 1000))

In [ ]:
#Crea un gráfico de dispersión y calcula la correlación entre las reseñas y las ventas.
plt.figure(figsize=(10, 6))
plt.scatter(wii_data['critic_score'], wii_data['venta_total'])
plt.title('Correlación entre Reseñas y Ventas en la plataforma Wii')
plt.xlabel('Puntuación del Crítico')
plt.ylabel('Ventas Totales')
plt.show()

corr_wii = wii_data['critic_score'].corr(wii_data['venta_total'])
print(f'Correlación entre Reseñas y Ventas en la plataforma Wii: {corr_wii}')

Se puede ver que hay una correlación positiva pero debil en cuanto a las ventas-críticas, éstas afectan pero no fuertemente a las ventas por lo que deben haber otros factores que también influyen en las ventas de los juegos de esta plataforma.

**3.9 Teniendo en cuenta tus conclusiones compara las ventas de los mismos juegos en otras plataformas.**

In [ ]:
platforms_of_interest = ['Wii', 'DS', 'X360']

platforms_data = games[games['platform'].isin(platforms_of_interest)]

# Agrupo los datos por plataforma y cuento los juegos
games_per_platform = platforms_data.groupby('platform')['name'].nunique().reset_index().rename(columns={'name': 'num_games'})

# Agrupo los datos por plataforma y sumo las ventas totales
sales_per_platform = platforms_data.groupby('platform')['venta_total'].sum().reset_index()

# Combino los datos en función de la plataforma
platform_info = pd.merge(games_per_platform, sales_per_platform, on='platform')

# Realizo un gráfico
plt.figure(figsize=(12, 6))
sns.barplot(x='platform', y='num_games', data=platform_info, color='skyblue', label='Número de Juegos')
plt.title('Relación entre Juegos Lanzados y Ventas por Plataforma')
plt.xlabel('Plataforma')
plt.ylabel('Cantidad / Ventas Totales')
plt.legend()
plt.xticks()
plt.show()



Echa un vistazo a la distribución general de los juegos por género. ¿Qué se puede decir de los géneros más rentables? ¿Puedes generalizar acerca de los géneros con ventas altas y bajas?

In [ ]:
#Agrupo los juegos por género y ventas
distribution_by_genre = games.groupby('genre')['venta_total'].sum().sort_values(ascending = False)
display(distribution_by_genre)

#Agrupo los juegos por genero y ventas por continente
distribution_by_genre = games.groupby('genre')[['na_sales', 'eu_sales', 'jp_sales']].sum()
display(distribution_by_genre)


**El top 5 de géneros de videojuegos a nivel mundial son:** acción, deportes, shooters, role-playing, platform.


**A nivel de continente top3:**

*America :* acción, deportes, shooters; *menos populares:* estrategia, aventura, puzzle.

*Europa:* accion, deportes,shooters; *menos populares:* estrategia, aventura, puzzle.

*Asia:* Role-playing, accion, deportes; *menos populares:* shooter, estrategia, aventura.

Los tipos de juegos más vendidos son juegos que las personas encuentran más entretenidos de jugar, que usan para recrearse y entretenerse. Los juegos de acción, deportes y de tirador(shooters) son los más populares actuamente.

**Paso 4: Crea un perfil de usuario para cada región para cada región (NA, UE, JP) determina:**

a) Las cinco plataformas principales. Describe las variaciones en sus cuotas de mercado de una región a otra.

b) Los cinco géneros principales. Explica la diferencia.

Si las clasificaciones de ESRB afectan a las ventas en regiones individuales.


In [ ]:
games.columns

In [ ]:
na_data = games[['platform', 'na_sales']].groupby('platform')['na_sales'].sum().reset_index()

# Ordeno las plataformas por ventas en América del Norte
top_platforms_na = na_data.sort_values(by='na_sales', ascending=False).head(5)

# Gráfico de barras para las cinco plataformas principales en América del Norte
plt.figure(figsize=(12, 6))
sns.barplot(x='platform', y='na_sales', data=top_platforms_na, palette='viridis')
plt.title('Top 5 Plataformas en América del Norte')
plt.xlabel('Plataforma')
plt.ylabel('Ventas en América del Norte (en millones)')
plt.show()

na_genre_data = games[['genre', 'na_sales']].groupby('genre')['na_sales'].sum().reset_index()

# Ordena los géneros por ventas en América del Norte
top_genres_na = na_genre_data.sort_values(by='na_sales', ascending=False).head(5)

# Gráfico de barras para los cinco géneros principales en América del Norte
plt.figure(figsize=(12, 6))
sns.barplot(x='genre', y='na_sales', data=top_genres_na, palette='magma')
plt.title('Top 5 Géneros en América del Norte')
plt.xlabel('Género')
plt.ylabel('Ventas en América del Norte (en millones)')
plt.show()

na_esrb_data = games[['rating', 'na_sales']].groupby('rating')['na_sales'].sum().reset_index()

# Gráfico de barras para las ventas por clasificación de ESRB en América del Norte
plt.figure(figsize=(12, 6))
sns.barplot(x='rating', y='na_sales', data=na_esrb_data, order=na_esrb_data.sort_values(by='na_sales', ascending=False)['rating'], palette='plasma')
plt.title('Impacto de las Clasificaciones de ESRB en Ventas en América del Norte')
plt.xlabel('Clasificación de ESRB')
plt.ylabel('Ventas en América del Norte (en millones)')
plt.show()




In [ ]:
na_concat = pd.concat([na_data, na_genre_data, na_esrb_data], axis=1)

# Mostrar el resultado
display(na_concat.head(10))


In [ ]:
eu_data = games[['platform', 'eu_sales']].groupby('platform')['eu_sales'].sum().reset_index()
# Ordeno las plataformas por ventas en Europa
top_platforms_eu = eu_data.sort_values(by = 'eu_sales', ascending = False).head(5)
# Gráfico de barras para las cinco plataformas principales en Europa
plt.figure(figsize=(12, 6))
sns.barplot(x= 'platform', y='eu_sales', data = top_platforms_eu, palette= 'viridis')
plt.title('Top 5 plataformas en Europa')
plt.xlabel('Plataforma')
plt.ylabel('Ventas en Europa (en millones)')
plt.show()

eu_genre_data = games[['genre', 'eu_sales']].groupby('genre')['eu_sales'].sum().reset_index()
# Ordeno los géneros por ventas en Europa
top_genres_eu = eu_genre_data.sort_values(by='eu_sales', ascending=False).head(5)
# Gráfico de barras para los cinco géneros principales en Europa
plt.figure(figsize=(12, 6))
sns.barplot(x='genre', y='eu_sales', data=top_genres_eu, palette='magma')
plt.title('Top 5 Géneros en Europa')
plt.xlabel('Género')
plt.ylabel('Ventas en Europa (en millones)')
plt.show()


eu_esrb_data = games[['rating', 'eu_sales']].groupby('rating')['eu_sales'].sum().reset_index()
# Gráfico de barras para las ventas por clasificación de ESRB en América del Norte
plt.figure(figsize=(12, 6))
sns.barplot(x='rating', y='eu_sales', data=eu_esrb_data, order=eu_esrb_data.sort_values(by='eu_sales', ascending=False)['rating'], palette='plasma')
plt.title('Impacto de las Clasificaciones de ESRB en Ventas en Europa')
plt.xlabel('Clasificación de ESRB')
plt.ylabel('Ventas en Europa (en millones)')
plt.show()

In [ ]:
eu_concat = pd.concat([eu_data, eu_genre_data, eu_esrb_data], axis=1)

# Mostrar el resultado
display(eu_concat.head(10))


In [ ]:

jp_data = games[['platform', 'jp_sales']].groupby('platform')['jp_sales'].sum().reset_index()
# Ordeno las plataformas por ventas en Japón
top_platforms_jp = jp_data.sort_values(by = 'jp_sales', ascending = False).head(5)
# Gráfico de barras para las cinco plataformas principales en Japon
plt.figure(figsize=(12, 6))
sns.barplot(x= 'platform', y='jp_sales', data = top_platforms_jp, palette= 'viridis')
plt.title('Top 5 plataformas en Japón')
plt.xlabel('Plataforma')
plt.ylabel('Ventas en Japón (en millones)')
plt.show()

jp_genre_data = games[['genre', 'jp_sales']].groupby('genre')['jp_sales'].sum().reset_index()
# Ordeno los géneros por ventas en Japón
top_genres_jp = jp_genre_data.sort_values(by='jp_sales', ascending=False).head(5)
# Gráfico de barras para los cinco géneros principales en Japón
plt.figure(figsize=(12, 6))
sns.barplot(x='genre', y='jp_sales', data=top_genres_jp, palette='magma')
plt.title('Top 5 Géneros en Japón')
plt.xlabel('Género')
plt.ylabel('Ventas en Japón (en millones)')
plt.show()


jp_esrb_data = games[['rating', 'jp_sales']].groupby('rating')['jp_sales'].sum().reset_index()
# Gráfico de barras para las ventas por clasificación de ESRB en América del Norte
plt.figure(figsize=(12, 6))
sns.barplot(x='rating', y='jp_sales', data=jp_esrb_data, order=jp_esrb_data.sort_values(by='jp_sales', ascending=False)['rating'], palette='plasma')
plt.title('Impacto de las Clasificaciones de ESRB en Ventas en Japón')
plt.xlabel('Clasificación de ESRB')
plt.ylabel('Ventas en Japón (en millones)')
plt.show()

In [ ]:
# Concatenar las Series horizontalmente
jp_concat = pd.concat([jp_data, jp_genre_data, jp_esrb_data], axis=1)

# Mostrar el resultado ordenado por ventas en Japón de manera descendente
display(jp_concat.head(10))


In [ ]:
all_concat = pd.concat([na_concat, eu_concat, jp_concat], axis= 1)
display(all_concat.head())

a) Las cinco plataformas principales. Describe las variaciones en sus cuotas de mercado de una región a otra.
En norteamerica las 5 principales x360, PS2, Wii, PS3, DS; en Europa PS2, PS3,X360, Wii, PS; Japón DS, PS, PS2, Snes, 3DS. Hay visibles variaciones en sus cuotas de mercado de una región a otra, puede estar la variación en lo popular que es la plataforma en ese continente, o el género del juego qué tan jugado es en ese lugar.


b) Los cinco géneros principales. Explica la diferencia.
Los generos principales son acción, deportes y shooter en norteamerica (na) y europa (eu) con diferencia en un género, la preferencia entre sus jugadores son más similares a diferencia de japón, que su género principal es role-playing, seguido de los generos que tambien son populares en na y eu (accion y deportes), puede deberse a diferencias culturales, diferencia en las edades etarias de los jugadores y preferencias.

*Si las clasificaciones de ESRB afectan a las ventas en regiones individuales.*

Si afectan, las clasificación más vendida es E (todos), y la manos vendida es A (adultos), 
dependiendo de la región se venden más juegos categoria E-T como en na o E-M como en eu.

**Paso 5: Prueba las siguientes hipótesis:**

— Las calificaciones promedio de los usuarios para las plataformas Xbox One y PC son las mismas.

— Las calificaciones promedio de los usuarios para los géneros de Acción y Deportes son diferentes.

Establece tu mismo el valor de umbral alfa.
Explica:
— Cómo formulaste las hipótesis nula y alternativa.
— Qué criterio utilizaste para probar las hipótesis y por qué.


— Las calificaciones promedio de los usuarios para las plataformas Xbox One y PC son las mismas.

In [ ]:
# Prueba las hipótesis
#H0: Las calificaciones promedio de los usuarios para las plataformas Xbox One y PC son las mismas.
#Ha: Las calificaciones promedio de los usuarios para las plataformas Xbox One y PC son diferentes.


# Calificaciones promedio para las plataformas Xbox One y PC
cal_xone = games[games['platform'] == 'XOne']['user_score']
cal_pc = games[games['platform'] == 'PC']['user_score']

alpha = 0.04
results = st.ttest_ind(cal_xone, cal_pc, equal_var = False, nan_policy = 'omit')

print('Valor p:', results.pvalue)

if results.pvalue < alpha:
    print('Rechazamos la hipótesis nula')
else:
    print('No podemos rechazar la hipótesis nula')


Primero establecí mi hipótesis nula basándome en lo estudiado ("H0 suele expresar la idea de que no hay diferencias entre los grupos"), luego, creé nuevas variables filtrando el DataFrame 'games' por plataforma (XOne y PC) y por el puntaje que le han dado los usuarios. Luego, utilizando la función st.ttest_ind para probar las hipótesis. Establecí un alpha de 0.04 porque quería ser un poco más rigurosa en ves de dejarlo en 0.05.

— Las calificaciones promedio de los usuarios para los géneros de Acción y Deportes son diferentes.

In [ ]:
# Prueba las hipótesis 2
#H0: Las calificaciones promedio de los usuarios para los géneros de acción y deporte son iguales.
#Ha: Las calificaciones promedio de los usuarios para los géneros de acción y deporte son diferentes.

# Calificaciones promedio para las plataformas Xbox One y PC
cal_action = games[games['genre'] == 'Action']['user_score']
cal_sports = games[games['genre'] == 'Sports']['user_score']

alpha = 0.04
results = st.ttest_ind(cal_action, cal_sports, equal_var=False, nan_policy = 'omit')

print('Valor p:', results.pvalue)

if results.pvalue < alpha:
    print('Rechazamos la hipótesis nula')
else:
    print('No podemos rechazar la hipótesis nula')

Para empezar establecí mi hipótesis nula, creé nuevas variables filtrando el DataFrame 'games' por género (action y sports) y por la calificación que le han dado los usuarios. Luego, utilicé la función st.ttest_ind para probar las hipótesis. 

**Conclusión general**
En este trabajo se buscó potenciar un trabajo más individualizado y la utilización de herramientas vistas durante el curso.
Se trabajó con datos de plataformas de videojuegos, ventas de los mismos clasificados por continente, género, clasificación por edad, entre otros. 
Llegué a la conclusión de que por continente hay grandes diferencias en los géneros que prefieren los usuarios a la hora de jugar, diferencias en ventas y exito en las distintas plaformas y que, año tras año van lanzandose nuevas plataformas y hay otras que dejan de trabajar.